# Introduction to LTspice.jl

a julia interface to [LTspice<sup>TM</sup>](http://www.linear.com/designtools/software/#LTspice)

<img src="circuit1.jpg">

In [1]:
using LTspice   # load the module

In [2]:
# create an simulation object 
circuitpath = "circuit1.asc"
circuit1 = LTspiceSimulation(circuitpath,tempdir=true)

LTspiceSimulation:
circuit path = C:\Users\Chris\AppData\Local\Temp\jl_8951.tmp\circuit1.asc

Parameters
rsource                   = 50.0
rload                     = 75.0

Measurements
Vload                    
Pload                    


Note: Use tempdir=true to create an LTspiceSimulation object with a copy of the circuit in a temporary directory

In [3]:
# circuit1 is a Dict like object
inputresistor = circuit1["rsource"]  # read a paramater by name

50.0

In [4]:
circuit1["rsource"] = 40.0  # write parameter by name.

40.0

In [5]:
loadpower = circuit1["Pload"] # measurements can also be read by name
# since a valid measurement was not available, this caused a simulation run

0.00567108

In [6]:
circuit1

LTspiceSimulation:
circuit path = C:\Users\Chris\AppData\Local\Temp\jl_8951.tmp\circuit1.asc

Parameters
rsource                   = 40.0
rload                     = 75.0

Measurements
Vload                     = 0.652174
Pload                     = 0.00567108

Last Run
time = 2016-10-22T22:35:49
duration = 0.007


In [7]:
circuit1["Pload"] = 1.0 # WRONG.  measurements cannot be writen.

LoadError: LoadError: measurements cannot be set.
while loading In[7], in expression starting on line 1

In [8]:
haskey(circuit1,"rload")  # can check for keys

true

In [9]:
keys(circuit1),values(circuit1) # arrays of keys and values
# unlike standard Dict, these will always be in order.

(String["rload","rsource","Pload","Vload"],[75.0,40.0,0.00567108,0.652174])

In [10]:
parameternames(circuit1)  # tuple of parameter names

("rsource","rload")

In [11]:
parametervalues(circuit1) # array of parameter values

2-element LTspice.ParameterValuesArray{Float64,1}:
 40.0
 75.0

In [12]:
measurementnames(circuit1) # tuple of measurement names

("Vload","Pload")

In [13]:
measurementvalues(circuit1) # array of measurement values.

2-element LTspice.MeasurementValuesArray{Float64,1}:
 0.652174  
 0.00567108

In [14]:
rsource = 1.0
rload = 2.0
(vload,pload) = circuit1(rsource,rload) # simulation can be called

2-element LTspice.MeasurementValuesArray{Float64,1}:
 0.666667
 0.222222

In [15]:
# call synthax is convenient for optimization
using Optim  # assumes Optim is installed
# find rload for max power
rsource = 50.0
result = optimize(rload -> -circuit1(rsource,rload)[2],10.0,100.0)
circuit1["rload"]

49.973239886404734

<img src="circuit2.jpg">

In [16]:
# create an LTspiceSimulation object for a stepped simulation
circuitpath = "circuit2.asc"
circuit2 = LTspiceSimulation(circuitpath,tempdir=true)

LTspiceSimulation:
circuit path = C:\Users\Chris\AppData\Local\Temp\jl_E77B.tmp\circuit2.asc

Parameters
rsource                   = 50.0

Measurements
Vload                    
Pload                    

Steps
rload                    
vin                      


In [17]:
# parameters are a Dict for stepped simualtion
inputresistance = circuit2["rsource"] 

50.0

In [18]:
loadvoltage = circuit2["Vload"]

7×3 Array{Float64,2}:
 0.285714  0.285714  0.285714
 0.375     0.375     0.375   
 0.444444  0.444444  0.444444
 0.5       0.5       0.5     
 0.545455  0.545455  0.545455
 0.583333  0.583333  0.583333
 0.615385  0.615385  0.615385

In [19]:
measurementvalues(circuit2)
# indexes are [inner_step, middle_step, outer_step, measurement]
# LTspice supports a maximum of 3 .step directives

7×3×2 LTspice.MeasurementValuesArray{Float64,3}:
[:, :, 1] =
 0.285714  0.285714  0.285714
 0.375     0.375     0.375   
 0.444444  0.444444  0.444444
 0.5       0.5       0.5     
 0.545455  0.545455  0.545455
 0.583333  0.583333  0.583333
 0.615385  0.615385  0.615385

[:, :, 2] =
 0.00408163  0.00408163  0.00408163
 0.0046875   0.0046875   0.0046875 
 0.00493827  0.00493827  0.00493827
 0.005       0.005       0.005     
 0.00495868  0.00495868  0.00495868
 0.00486111  0.00486111  0.00486111
 0.00473373  0.00473373  0.00473373

In [20]:
stepnames(circuit2) # tuple of step names

("rload","vin")

In [21]:
stepvalues(circuit2) # tuple of arrays of step values

([20.0,30.0,40.0,50.0,60.0,70.0,80.0],[1.0,2.0,3.0])

In [22]:
# use perlineiterator to write circuit2 to csv with headers
open("test.csv",false,true,true,false,false) do io
    writecsv(io,perlineiterator(circuit2,header=true))
end

In [23]:
# this is what it looks like
for line in perlineiterator(circuit2,header=true)
    println(line)
end

("rload","vin","rsource","Vload","Pload")
(20.0,1.0,50.0,0.285714,0.00408163)
(30.0,1.0,50.0,0.375,0.0046875)
(40.0,1.0,50.0,0.444444,0.00493827)
(50.0,1.0,50.0,0.5,0.005)
(60.0,1.0,50.0,0.545455,0.00495868)
(70.0,1.0,50.0,0.583333,0.00486111)
(80.0,1.0,50.0,0.615385,0.00473373)
(20.0,2.0,50.0,0.285714,0.00408163)
(30.0,2.0,50.0,0.375,0.0046875)
(40.0,2.0,50.0,0.444444,0.00493827)
(50.0,2.0,50.0,0.5,0.005)
(60.0,2.0,50.0,0.545455,0.00495868)
(70.0,2.0,50.0,0.583333,0.00486111)
(80.0,2.0,50.0,0.615385,0.00473373)
(20.0,3.0,50.0,0.285714,0.00408163)
(30.0,3.0,50.0,0.375,0.0046875)
(40.0,3.0,50.0,0.444444,0.00493827)
(50.0,3.0,50.0,0.5,0.005)
(60.0,3.0,50.0,0.545455,0.00495868)
(70.0,3.0,50.0,0.583333,0.00486111)
(80.0,3.0,50.0,0.615385,0.00473373)
